<a href="https://colab.research.google.com/github/aianytyme/gpt3-for-capital-markets/blob/main/gpt3_tune_using_financial_10k_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPT3-Tune Using Financial 10K Data**

1.   Setup
2.   Load Data 
3.   Tune
4.   Clean     

# Setup

In [6]:
!pip install python-dotenv
!pip install --upgrade jsonlines
!pip install --upgrade openai
!pip install wandb

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 181 kB 70.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e273f36e2dcc0146fb09ad94ea998c59b23a90709cb747a50f77d866ab01bbe4
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [7]:
from google.colab import drive
drive.mount('/content/drive/')
PATH= r'/content/drive/MyDrive/Projects/GPT-3/'
print("Path:{}".format(PATH))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Path:/content/drive/MyDrive/Projects/GPT-3/


In [8]:
from os import environ
from dotenv import load_dotenv
load_dotenv(PATH+'.env')
print("Env File to load:{}".format(PATH+'.env'))

Env File to load:/content/drive/MyDrive/Projects/GPT-3/.env


# Load SEC 10K Data

In [16]:
import os
import json
import jsonlines
import pandas as pd

In [43]:
file_to_load = PATH + 'data/salesforce.jsonl'
print("file_to_load:{}".format(file_to_load))

#with open(file_to_load, 'r') as json_file:
#  data = json.load(json_file)

with open(file_to_load, 'r') as json_file:
  for line in json_file:
    print(line)

#df = pd.DataFrame({'count': data})
#df.head(10)


file_to_load:/content/drive/MyDrive/Projects/GPT-3/data/salesforce.jsonl
{"prompt": "", "completion": "Company : SALESFORCE.COM, INC. : ITEM 1. BUSINESS  Overview: Salesforce is a global leader in customer relationship management  �CRM�  technology that brings companies and their customers together. With our Customer 360 platform, we deliver a single source of truth, connecting customer data across systems, apps and devices to help companies sell, service, market and conduct commerce from anywhere. Since its founding in 1999, Salesforce has pioneered innovations in cloud, mobile, social, analytics and artificial intelligence  �AI� , enabling companies of every size and industry to transform their businesses in the all-digital, work-from-anywhere era.  Our Customer 360 service offerings are designed to be flexible, scalable and easy to use. They can generally be configured easily, rapidly deployed and integrated with other platforms and enterprise applications. We sell to businesses wor

# Tune the OpenAI model

**Create a new model using the JSONL** 

In [48]:
!openai api fine_tunes.create -t $file_to_load -m curie --suffix "gpt3-for-capital-markets"

Found potentially duplicated files with name 'salesforce.jsonl', purpose 'fine-tune' and size 6579 bytes
file-pLl5AlroWYAAYTbIb06Rr6Rd
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: file-pLl5AlroWYAAYTbIb06Rr6Rd
Reusing already uploaded file: file-pLl5AlroWYAAYTbIb06Rr6Rd
Created fine-tune: ft-wc5PX33YH1OqZ5Nfgqehotql
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-05-02 23:56:25] Created fine-tune: ft-wc5PX33YH1OqZ5Nfgqehotql
[2022-05-02 23:56:28] Fine-tune costs $0.01
[2022-05-02 23:56:29] Fine-tune enqueued. Queue number: 0
[2022-05-02 23:56:33] Fine-tune started
[2022-05-02 23:57:21] Completed epoch 1/4
[2022-05-02 23:57:22] Completed epoch 2/4
[2022-05-02 23:57:23] Completed epoch 3/4
[2022-05-02 23:57:24] Completed epoch 4/4
[2022-05-02 23:57:49] Uploaded model: curie:ft-the-orange-pencil:gpt3-for-capital-markets-2022-05-02-23-57-48
[2022-05-02 23:57:52] Uplo

**Test the new model**

In [55]:
CUSTOM_GPT3_ENGINE='curie:ft-the-orange-pencil:gpt3-for-capital-markets-2022-05-02-23-57-48'
!openai api completions.create -m $CUSTOM_GPT3_ENGINE -p "Summarize this\n\n "

InvalidRequestError: ignored

**Analyze the fine-tuned model**

In [46]:
!echo $CUSTOM_GPT3_ENGINE
!openai api fine_tunes.results -i $CUSTOM_GPT3_ENGINE

curie:ft-the-orange-pencil:gpt3-for-capital-markets-2022-05-02-23-21-07
[organization=nomura-1] Error: No fine-tune job: curie:ft-the-orange-pencil:gpt3-for-capital-markets-2022-05-02-23-21-07 (HTTP status code: 404)


In [56]:
!openai api models.delete -i $CUSTOM_GPT3_ENGINE
#!openai api models.delete -i curie:ft-the-orange-pencil:gpt3-for-capital-markets-2022-05-02-23-15-39

{
  "deleted": true,
  "id": "curie:ft-the-orange-pencil:gpt3-for-capital-markets-2022-05-02-23-15-39",
  "object": "model"
}
